In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Оригинальная архитектура сети VGG19 представлена 16 свёрточными слоями в 5 блоках свёртки, после каждого из которых применяется MaxPooling. В то же время Гатис и др. в своей статье отмечают, что использование AvgPooling вместо MaxPooling улучшает градиент и делает результирующие изображения более привлекательными с точки зрения восприятия человеком.

На разных этапах свёртки информация о входном изображении отличается, увеличивается число накопленных признаков благодаря росту количества применённых фильтров. Одновременно с этим уменьшается разрешение самого изображения после очередного применения downsampling-механизма.

Авторы статьи предприняли попытку визуализировать накопленную информацию на разных слоях CNN. Входное изображение было по очереди воссоздано из первого слоя каждого свёрточного блока. В результате удалось выяснить, что реконструированное изображение с первых слоёв первых трёх блоков почти идентично исходному, а далее информация об отдельных пикселях начинает теряться, но при этом сохраняется "высокоуровневая" информация об объектах на изображении, т.е. об их форме, взаимном расположении и т.п.

Кроме того авторы статьи предприняли различные попытки воссоздания стиля изображения. Для сохранения информации о стиле считалась корреляция между всеми выявленными признаками, которые были найдены фильтрами на разных слоях CNN. Далее были использованы 5 различных наборов корреляций, полученных соответственно с первых слоёв следующих свёрточных блоков: 1; 1 и 2; 1-3; 1-4; 1-5. Авторам удалось выяснить, что использование нового дополнительного слоя для реконструкции стиля постепенно увеличивает масштаб отрисовки отдельного признака, при этом информация о взаимном расположении данных признаков постепенно утрачивается.

Таким образом самым рациональным подходом для Style Transfer алгоритма будет использование карты признаков одного из "верхних" слоёв CNN для переноса контента на результирующее изображение, и использование нескольких "глубоких" слоёв CNN для переноса стиля. Авторами статьи были использованы первые слои всех пяти блоков для сохранения стиля и один слой четвёртого блока для переноса контента.

Для повторения некоторых экспериментов, описанных и проведённых Гатисом и др., определим параметризованный конструктор, для возможности создания сетей с различным pooling-типом и параметризованную функцию forward(), чтобы иметь возможность определять выходы каких слоёв мы будем использовать при переносе стиля.

In [ ]:
class VGG_nst(nn.Module):
    def __init__(self, pooling = None):
        super(VGG,self).__init__()

        if pooling != 'avg' and pooling != 'max':
            raise BaseException("Неправильно указан pooling-тип. " +
                                "Допустимые значения: avg, max.")

        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        

        if pooling is 'avg':
            self.pool_1 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_2 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_3 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_4 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_5 = nn.AvgPool2d(kernel_size=2, stride=2)
        else:
            self.pool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_2 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_3 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_4 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_5 = nn.MaxPool2d(kernel_size=2, stride=2)

        
    def forward(self, x, out_layers):
        out['conv1_1'] = F.relu(self.conv1_1(x))
        out['conv1_2'] = F.relu(self.conv1_2(out['conv1_1']))
        out['pool_1'] = self.p1(out['conv1_2'])
        
        out['conv2_1'] = F.relu(self.conv2_1(out['pool_1']))
        out['conv2_2'] = F.relu(self.conv2_2(out['conv2_1']))
        out['pool_2'] = self.p2(out['conv2_2'])
        
        out['conv3_1'] = F.relu(self.conv3_1(out['pool_2']))
        out['conv3_2'] = F.relu(self.conv3_2(out['conv3_1']))
        out['conv3_3'] = F.relu(self.conv3_3(out['conv3_2']))
        out['conv3_4'] = F.relu(self.conv3_4(out['conv3_3']))
        out['pool_3'] = self.p3(out['conv3_4'])
        
        out['conv4_1'] = F.relu(self.conv4_1(out['pool_3']))
        out['conv4_2'] = F.relu(self.conv4_2(out['conv4_1']))
        out['conv4_3'] = F.relu(self.conv4_3(out['conv4_2']))
        out['conv4_4'] = F.relu(self.conv4_4(out['conv4_3']))
        out['pool_4'] = self.p4(out['conv4_4'])
        
        out['conv5_1'] = F.relu(self.conv5_1(out['pool_4']))
        out['conv5_2'] = F.relu(self.conv5_2(out['conv5_1']))
        out['conv5_3'] = F.relu(self.conv5_3(out['conv5_2']))
        out['conv5_4'] = F.relu(self.conv5_4(out['conv5_3']))
        out['pool_5'] = self.p5(out['conv5_4'])

        return [out[layer] for layer in out_layers]